# Twin study reveals non-heritable immune perturbations in multiple sclerosis
# CITE-Seq analysis

Reference:
 Ingelfinger F, Gerdes LA, Kavaka V, Krishnarajah S, Friebel E, Galli E, Zwicky P, Furrer R, Peukert C, Dutertre CA, Eglseer KM, Ginhoux F, Flierl-Hecht A, Kümpfel T, De Feo D, Schreiner B, Mundt S, Kerschensteiner M, Hohlfeld R, Beltrán E†, Becher B†. Twin study reveals non-heritable immune perturbations in multiple sclerosis. Nature (in press)

Code authors:
 Vladyslav Kavaka, Florian Ingelfinger, Klara Magdalena Eglseer, Eduardo Beltrán

In [ ]:
library(devtools)
library(Seurat)
library(dplyr)
library(Matrix)
library(tidyr)
library(limma)
library(ggplot2)
library(ggpubr)
library(plyr)
library(purrr)
library(ggrepel)
library(gprofiler2)
library(scales)
library(ggthemes)
library(reshape2)
library(rstatix)
library(tibble)
library(pheatmap)
library(stringr)
library(lmtest)
library(patchwork)

In [ ]:
options(repr.plot.width=11, repr.plot.height=11)

# Read 10X data

In [ ]:
## Read 10X data:
matrix_dir = "data/filtered_feature_bc_matrix/"
data <- Read10X(data.dir = matrix_dir)
## Create Seurat object
pbmc <- CreateSeuratObject (counts = data$'Gene Expression', min.cells = 3, min.features = 200, project = "MS_TWIN_STUDY")

In [ ]:
# The number of features and UMIs (nFeature_RNA and nCount_RNA) are automatically calculated for every object by Seurat.
# For non-UMI data, nCount_RNA represents the sum of the non-normalized values within a cell
# We calculate the percentage of mitochondrial features here and store it in object metadata as `percent.mito`.
# We use raw count data since this represents non-transformed and non-log-normalized counts
# The % of UMI mapping to MT-features is a common scRNA-seq QC metric.
mito.features <- grep(pattern = "^MT-", x = rownames(x = pbmc), value = TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts'))
pbmc[['percent.mito']] <- percent.mito
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3, pt.size = 0.01)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used for anything 
# calculated by the object, i.e. columns in object metadata, PC scores etc.
# Since there is a rare subset of cells with an outlier level of high mitochondrial percentage
# and also low UMI content, we filter these as well
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mito")

In [ ]:
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
# We filter out cells that have unique feature counts over 6,000 or less than 500
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 500 & nFeature_RNA < 6000 & percent.mito < '0.15')

## Create a column with sample numbers

In [ ]:
# Get batches based on cell names
samples_batches <- sapply(colnames(GetAssayData(object = pbmc, slot = "counts")),
                      FUN=function(x){substr(x,18,19)})

# Turn to numbers and add cell names to them
samples_batches <- as.numeric(as.character(samples_batches))
names(samples_batches) <- colnames(GetAssayData(object = pbmc, slot = "counts"))

sample.effect <- samples_batches

In [ ]:
pbmc <- AddMetaData(pbmc, sample.effect, "sample.effect")

# Adding the sample information

In [ ]:
samplenames <- read.csv(file = './sample_names.csv')
#sort column - p5 = CD4+ cells, p8 = CD11b+ cells

In [ ]:
#add Sample, twin and sorting information into the meta.data
for(i in 1:nrow(pbmc@meta.data)){
    pbmc@meta.data$sample[i] <- filter(samplenames, Number == pbmc@meta.data$sample.effect[i])$Sample
    pbmc@meta.data$twin[i] <- filter(samplenames, Number == pbmc@meta.data$sample.effect[i])$twin
    pbmc@meta.data$sort[i] <- filter(samplenames, Number == pbmc@meta.data$sample.effect[i])$sort
}

# Normalize and find variable features

In [ ]:
pbmc <- NormalizeData(object = pbmc, normalization.method = "LogNormalize", scale.factor = 1e4)

In [ ]:
pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", nfeatures = 2000)
length(x = VariableFeatures(object = pbmc))
#remove TCR variable chains before running the PCA
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV",  x = rownames(x = pbmc), value = TRUE)
VariableFeatures(object = pbmc) <- VariableFeatures(object = pbmc)[!(VariableFeatures(object = pbmc)%in%markers.remove)]
length(VariableFeatures(object = pbmc))

# Scale and run PCA

In [ ]:
pbmc <- ScaleData(pbmc, features = VariableFeatures(pbmc))
pbmc <- RunPCA(pbmc, features = VariableFeatures(object = pbmc))

DimPlot(pbmc)

In [ ]:
# ProjectDim scores each feature in the dataset (including features not included in the PCA) based on their correlation 
# with the calculated components. Though we don't use this further here, it can be used to identify markers that 
# are strongly correlated with cellular heterogeneity, but may not have passed through variable feature selection. 
# The results of the projected PCA can be explored by setting `projected = TRUE`in the functions above
pbmc <- ProjectDim(object = pbmc)

In [ ]:
#explore the Elbow and the PCA Heatmaps
ElbowPlot(object = pbmc)
for (i in 1:25){
    print(DimHeatmap(object = pbmc, dims = i, cells = 500, balanced = TRUE))
}

# Cluster the cells

In [ ]:
pbmc <- FindNeighbors(object = pbmc, dims = 1:9)
pbmc <- FindClusters(object = pbmc, resolution = 0.5)

## Run UMAP

In [ ]:
pbmc <- RunUMAP(pbmc, dims = 1:9)

In [ ]:
#umap, dims 1:9, res. 0.5
DimPlot(pbmc, reduction = 'umap', label = TRUE)
DimPlot(pbmc, reduction = 'umap', label = TRUE, group.by = 'sample.effect')
DimPlot(pbmc, reduction = 'umap', label = TRUE, group.by = 'twin')
DimPlot(pbmc, reduction = 'umap', label = TRUE, group.by = 'sort')

# Adding Ab information

In [ ]:
#rename the AB names
rownames(data[[2]]) <- gsub('_TotalSeqC', '-Ab', x = rownames(data[[2]]))
rownames(data[[2]]) <- gsub('_', '-', x = rownames(data[[2]]))
rownames(data[[2]])
Ab <- data[[2]]
Ab <- Ab[, (colnames(Ab) %in% rownames(pbmc@meta.data))]

In [ ]:
#Create protein expression assay
pbmc[["Ab"]] <- CreateAssayObject(counts = Ab)

#Normalize the Ab assay
pbmc <- NormalizeData(object = pbmc, assay = "Ab", normalization.method = "CLR")
pbmc <- ScaleData(pbmc, assay = "Ab")

# Adding TCR information

In [ ]:
tcr_folder = 'tcr_data/TR_84_P5'

In [ ]:
#First we need to create TCR file for sample 84
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_84 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_139'

In [ ]:
#Create TCR file for sample 139
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 3, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_139 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_140_P5'

In [ ]:
#Create TCR file for sample 140
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 4, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_140 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_157_P5'

In [ ]:
#Create TCR file for sample 157
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 6, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_157 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_158_P5'

In [ ]:
#Create TCR file for sample 158
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 8, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_158 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_165_P5'

In [ ]:
#Create TCR file for sample 165
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 10, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_165 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_166_P5'

In [ ]:
#Create TCR file for sample 166
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 12, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_166 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_182_P5'

In [ ]:
#Create TCR file for sample 182
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 14, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_182 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_323_P5'

In [ ]:
#Create TCR file for sample 323
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 16, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_323 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_324_P5'

In [ ]:
#Create TCR file for sample 324
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 18, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_324 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_327_P5'

In [ ]:
#Create TCR file for sample 327
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 20, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_327 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_328_P5'

In [ ]:
#Create TCR file for sample 328
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 22, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_328 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_343_P5'

In [ ]:
#Create TCR file for sample 343
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 24, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_343 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_344_P5'

In [ ]:
#Create TCR file for sample 344
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 26, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_344<- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_351_P5'

In [ ]:
#Create TCR file for sample 351
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 28, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_351 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_352_P5'

In [ ]:
#Create TCR file for sample 352
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 30, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_352 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_353_P5'

In [ ]:
#Create TCR file for sample 353
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 32, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_353 <- tcr

In [ ]:
tcr_folder = 'tcr_data/TR_354_P5'

In [ ]:
#Create TCR file for sample 354
tcr <- read.csv(paste(tcr_folder, '/outs/', "filtered_contig_annotations.csv", sep=""))
tcr$barcode <- gsub("-1", paste('-', 34, sep = ''), tcr$barcode) 
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(tcr_folder, '/outs/', "clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"

head(tcr)
tcr_354 <- tcr

In [ ]:
#combine TCR data together
tcr.combined <- rbind(tcr_84, tcr_139, tcr_140, tcr_157, tcr_158, tcr_165, tcr_166, tcr_182, tcr_323, tcr_324, tcr_327, tcr_328, tcr_343, tcr_344, tcr_351, tcr_352, tcr_353, tcr_354)

In [ ]:
#divide in TRA and TRB subset:
for (k in 1:nrow(tcr.combined)){
  if(startsWith(tcr.combined$TCR1[k], 'TRB:')){
    tcr.combined$TCR1B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR1[k], 'TRA:')){
    tcr.combined$TCR1A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR2[k], 'TRB:')){
    tcr.combined$TCR2B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2B[k] <- 'FALSE'}
        if(startsWith(tcr.combined$TCR2[k], 'TRA:')){
    tcr.combined$TCR2A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR3[k], 'TRB:')){
    tcr.combined$TCR3B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3B[k] <- 'FALSE'}
     if(startsWith(tcr.combined$TCR3[k], 'TRA:')){
    tcr.combined$TCR3A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR4[k], 'TRB:')){
    tcr.combined$TCR4B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR4[k], 'TRA:')){
    tcr.combined$TCR4A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4A[k] <- 'FALSE'}
}

In [ ]:
#delete the columns after division in TRA and TRB
tcr.combined$TCR1 <- NULL
tcr.combined$TCR2 <- NULL
tcr.combined$TCR3 <- NULL
tcr.combined$TCR4 <- NULL

In [ ]:
#add the TCR into the meta.data and replace the NA with 'FALSE'
pbmc <- AddMetaData(object = pbmc, metadata = tcr.combined)
md = pbmc@meta.data # First, let's get the meta data
i <- sapply(md, is.factor) # Identify all factor variables in your data
md[i] <- lapply(md[i], as.character) # Convert factors to character variables
md[is.na(md)] <- "FALSE" # Replace NA with "FALSE"
md[i] <- lapply(md[i], as.factor) # Convert character columns back to factors
pbmc@meta.data = md #Insert it back

In [ ]:
DimPlot(pbmc, reduction = 'umap', label = TRUE)
DimPlot(pbmc, reduction = 'umap', group.by = 'TCR_frequency', label = TRUE) + NoLegend()
DimPlot(pbmc, reduction = 'umap', group.by = 'TCR_chain', label = TRUE)

# Subset the T cells, clustering and integration 

In [ ]:
DefaultAssay(pbmc) <- 'RNA'
Idents(pbmc) <- 'sort'
#subset the T cells by subsetting P5 sort (CD4 + Cells)
tcells <- subset(pbmc, idents = 'p5')
tcells

In [ ]:
#subset only the ones with the TCR information
Idents(tcells) <- 'TCR_chain'
tcells <- subset(tcells, idents = 'FALSE', invert = TRUE)
tcells

In [ ]:
#find variable features 
tcells <- FindVariableFeatures(tcells, selection.method = "vst", nfeatures = 2000)
length(x = VariableFeatures(object = tcells))
#remove the TCR variable chain genes before reclustering
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV",  x = rownames(x = tcells), value = TRUE)
VariableFeatures(object = tcells) <- VariableFeatures(object = tcells)[!(VariableFeatures(object = tcells)%in%markers.remove)]
length(VariableFeatures(object = tcells))

In [ ]:
#scale data, run PCA
tcells <- ScaleData(tcells, features = VariableFeatures(object = tcells), vars.to.regress = c("nCount_RNA", "percent.mito"))
tcells <- RunPCA(tcells, features = VariableFeatures(object = tcells))

In [ ]:
#explore the Elbow and the PCA Heatmaps
ElbowPlot(object = tcells)
for (i in 1:25){
    print(DimHeatmap(object = tcells, dims = i, cells = 500, balanced = TRUE))
}

## Cluster the cells

In [ ]:
tcells <- FindNeighbors(object = tcells, dims = 1:21)

In [ ]:
tcells <- FindClusters(object = tcells, resolution = 0.6)

## Run UMAP

In [ ]:
tcells <- RunUMAP(tcells, dims = 1:21)

In [ ]:
#vst, umap, dims 21, res. 0.6
DimPlot(tcells, reduction = 'umap', label = TRUE)
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'sample')
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'TCR_frequency')

## Cluster markers

In [ ]:
DefaultAssay(tcells) <- 'RNA'

In [ ]:
featurestcells <- rownames(tcells)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV", x = rownames(tcells), value = TRUE)
featurestcells <- featurestcells[!(featurestcells%in%markers.remove)]
tcells.markers1 <- FindAllMarkers(object = tcells, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featurestcells)

In [ ]:
#sort top 30 first markers per cluster
tcells.markers1_sorted <- c()
for (i in 1:length(levels(tcells.markers1$cluster))){
    tcells.markers1_level <- filter(tcells.markers1, cluster == levels(tcells.markers1$cluster)[i])
    tcells.markers1_level <- tcells.markers1_level[order(-tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_level <- tcells.markers1_level[1:30, ]
    tcells.markers1_level <- tcells.markers1_level[!is.na(tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_sorted <- rbind(tcells.markers1_sorted, tcells.markers1_level)
    }
tcells.markers1_sorted_top30 <- tcells.markers1_sorted
write.csv(tcells.markers1_sorted_top30, file = './tcells_top30_first.csv')

In [ ]:
#compute the average expression for heatmaps
cluster.averages_tcells <- AverageExpression(tcells, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
#sort top5 first markers per cluster
tcells.markers1_sorted <- c()
for (i in 1:length(levels(tcells.markers1$cluster))){
    tcells.markers1_level <- filter(tcells.markers1, cluster == levels(tcells.markers1$cluster)[i])
    tcells.markers1_level <- tcells.markers1_level[order(-tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_level <- tcells.markers1_level[1:5, ]
    tcells.markers1_level <- tcells.markers1_level[!is.na(tcells.markers1_level$avg_log2FC), ]
    tcells.markers1_sorted <- rbind(tcells.markers1_sorted, tcells.markers1_level)
    }
tcells.markers1_sorted_top5 <- tcells.markers1_sorted

In [ ]:
#plot top5 markers per cluster in heatmap
DoHeatmap(cluster.averages_tcells, features = tcells.markers1_sorted_top5$gene)+ theme(text = element_text(size = 20, face = "bold"))
#plot the nfeatures per cell in VlnPlot (helpuf for sorting out low quality cells)
VlnPlot(tcells, features = 'nFeature_RNA', pt.size = 0.01)

## Subset out the low quality cells and contamination, second round of reclustering:

In [ ]:
#clusters 7 and 10 are contaminated and low quality clusters. Subset them out
write.csv(filter(tcells@meta.data, seurat_clusters == 7 | seurat_clusters == 10), file = './lowquality_contamination_cd4.csv')
tcells <- subset(tcells, idents = c(7, 10), invert = TRUE)
tcells

In [ ]:
#find variable features
tcells <- FindVariableFeatures(tcells, selection.method = "vst", nfeatures = 1500)
length(x = VariableFeatures(object = tcells))
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV",  x = rownames(x = tcells), value = TRUE)
VariableFeatures(object = tcells) <- VariableFeatures(object = tcells)[!(VariableFeatures(object = tcells)%in%markers.remove)]
length(VariableFeatures(object = tcells))

In [ ]:
tcells <- ScaleData(tcells, features = VariableFeatures(object = tcells), vars.to.regress = c("nCount_RNA", "percent.mito"))
tcells <- RunPCA(tcells, features = VariableFeatures(object = tcells))

In [ ]:
#explore the Elbow and the PCA Heatmaps
ElbowPlot(object = tcells)
for (i in 1:25){
    print(DimHeatmap(object = tcells, dims = i, cells = 500, balanced = TRUE))
}

## Cluster the cells

In [ ]:
tcells <- FindNeighbors(object = tcells, dims = 1:20)

In [ ]:
tcells <- FindClusters(object = tcells, resolution = 0.5)

## Run UMAP

In [ ]:
tcells <- RunUMAP(tcells, dims = 1:20)

In [ ]:
#vst, umap, dims 20, res. 0.5
DimPlot(tcells, reduction = 'umap', label = TRUE)
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'sample')
DimPlot(tcells, reduction = 'umap', label = TRUE, group.by = 'TCR_frequency')

## As seen in the previous UMAPs, the batch effect was detected. Integrate the T cells data (rPCA based method)

In [ ]:
#create a list of samples for integration
tcells.list <- SplitObject(tcells, split.by = "sample.effect")
tcells.list

In [ ]:
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV",  x = rownames(x = tcells), value = TRUE)
tcells.list <- lapply(X = tcells.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 5000)
})

#remove the variable chains before the integration
for (i in 1:length(tcells.list)){
     VariableFeatures(tcells.list[[i]]) <- VariableFeatures(object = tcells.list[[i]])[!(VariableFeatures(object = tcells.list[[i]])%in%markers.remove)]
}

In [ ]:
#select the features for integration
features <- SelectIntegrationFeatures(object.list = tcells.list, nfeatures = 5000)

In [ ]:
tcells.list <- lapply(X = tcells.list, FUN = function(x) {
    x <- ScaleData(x, features = features, verbose = FALSE)
    x <- RunPCA(x, features = features, verbose = FALSE)
})

In [ ]:
tcells.anchors <- FindIntegrationAnchors(object.list = tcells.list, anchor.features = features, reduction = "rpca")

In [ ]:
tcells.integrated <- IntegrateData(anchorset = tcells.anchors)

## Cluster the integrated file

In [ ]:
DefaultAssay(tcells.integrated) <- 'integrated'

In [ ]:
tcells.integrated <- FindVariableFeatures(tcells.integrated, selection.method = "vst", nfeatures = 1500)

In [ ]:
tcells.integrated <- ScaleData(tcells.integrated, features = VariableFeatures(object = tcells.integrated), vars.to.regress = c("nCount_RNA", "percent.mito"))
tcells.integrated <- RunPCA(tcells.integrated, features = VariableFeatures(object = tcells.integrated))

In [ ]:
ElbowPlot(object = tcells.integrated, ndims = 50)

In [ ]:
for (i in 1:25){
    print(DimHeatmap(object = tcells.integrated, dims = i, cells = 500, balanced = TRUE))
}

In [ ]:
tcells.integrated

In [ ]:
DefaultAssay(tcells) <- 'integrated'

In [ ]:
tcells <- FindNeighbors(object = tcells, dims = 1:11)

In [ ]:
#increase the resolution to gain a better in depth insight into the t cells populations
tcells <- FindClusters(object = tcells, resolution = 1.6)

In [ ]:
Idents(tcells) <- 'seurat_clusters'
options(repr.plot.width = 13, repr.plot.height = 11)
DimPlot(tcells, reduction = "umap", label = TRUE, label.size = 10, repel = TRUE, pt.size = 0.7) + 
theme(axis.line = element_line(size=1),
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      axis.ticks = element_line(size=1),
      legend.text=element_text(size=20))
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
VlnPlot(tcells, features = 'nFeature_RNA', pt.size = 0.01)

## Cluster markers

In [ ]:
DefaultAssay(tcells.integrated) <- 'RNA'

In [ ]:
featurestcells.integrated <- rownames(tcells.integrated)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV", x = rownames(tcells.integrated), value = TRUE)
featurestcells.integrated <- featurestcells.integrated[!(featurestcells.integrated%in%markers.remove)]
tcells.integrated.markers1 <- FindAllMarkers(object = tcells.integrated, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featurestcells.integrated)

In [ ]:
#sort top 30 markers
tcells.integrated.markers1_sorted <- c()
for (i in 1:length(levels(tcells.integrated.markers1$cluster))){
    tcells.integrated.markers1_level <- filter(tcells.integrated.markers1, cluster == levels(tcells.integrated.markers1$cluster)[i])
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[order(-tcells.integrated.markers1_level$avg_log2FC), ]
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[1:30, ]
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[!is.na(tcells.integrated.markers1_level$avg_log2FC), ]
    tcells.integrated.markers1_sorted <- rbind(tcells.integrated.markers1_sorted, tcells.integrated.markers1_level)
    }
tcells.integrated.markers1_sorted_top30 <- tcells.integrated.markers1_sorted
tcells.integrated.markers1_sorted_top30
write.csv(tcells.integrated.markers1_sorted_top30, file = './tcells.integrated_top30_first.csv')

In [ ]:
cluster.averages_tcells.integrated <- AverageExpression(tcells.integrated, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
#sort top 5 markers
tcells.integrated.markers1_sorted <- c()
for (i in 1:length(levels(tcells.integrated.markers1$cluster))){
    tcells.integrated.markers1_level <- filter(tcells.integrated.markers1, cluster == levels(tcells.integrated.markers1$cluster)[i])
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[order(-tcells.integrated.markers1_level$avg_log2FC), ]
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[1:5, ]
    tcells.integrated.markers1_level <- tcells.integrated.markers1_level[!is.na(tcells.integrated.markers1_level$avg_log2FC), ]
    tcells.integrated.markers1_sorted <- rbind(tcells.integrated.markers1_sorted, tcells.integrated.markers1_level)
    }
tcells.integrated.markers1_sorted_top5 <- tcells.integrated.markers1_sorted
tcells.integrated.markers1_sorted_top5

In [ ]:
DoHeatmap(cluster.averages_tcells.integrated, features = tcells.integrated.markers1_sorted_top5$gene)+ theme(text = element_text(size = 20, face = "bold"))

## combine clusters, remove low quality cells

In [ ]:
### combine clusters 7/2, 10/15/6/11, 13/14 due to their high similarity
#create a new subcluster column
tcells.integrated@meta.data$recluster <- tcells.integrated@meta.data$seurat_clusters

tcells.integrated@meta.data$recluster[tcells.integrated@meta.data$seurat_clusters == 7] <- 2

tcells.integrated@meta.data$recluster[tcells.integrated@meta.data$seurat_clusters == 10] <- 6
tcells.integrated@meta.data$recluster[tcells.integrated@meta.data$seurat_clusters == 11] <- 6
tcells.integrated@meta.data$recluster[tcells.integrated@meta.data$seurat_clusters == 15] <- 6

tcells.integrated@meta.data$recluster[tcells.integrated@meta.data$seurat_clusters == 14] <- 13

### remove cluster 9, 17 and 19 as low quality cells cluster
tcells.integrated <- subset(tcells.integrated, idents = c(9, 17, 19), invert = TRUE)

In [ ]:
Idents(tcells) <- 'recluster'
options(repr.plot.width = 13, repr.plot.height = 11)
DimPlot(tcells, reduction = "umap", label = TRUE, label.size = 10, repel = TRUE, pt.size = 0.7) + 
theme(axis.line = element_line(size=1),
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      axis.ticks = element_line(size=1),
      legend.text=element_text(size=20))
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
#explore some morekers from the DGE analysis and AB assay
DefaultAssay(tcells) <- 'RNA'
markers <- c('CD27', 'CD7', 'CCR7', 'CCR7-Ab', 'CCR7', 'CD45RA-Ab', 'CD45RO-Ab', 'CD25-Ab', 'IL2RA', 'CXCR4', 'IL2', 'CD45RA-Ab', 'CD45RO-Ab', 'CCR7', 'IFI6', 'FOXP3', 'CX3CR1', 'CCL4')
for(i in 1:length(markers)){
    print(VlnPlot(tcells, features = markers[i], pt.size = 0.01, sort = TRUE) +
        theme(axis.line = element_line(size=1),
              text = element_text(size = 20),
              axis.text = element_text(size = 20),
              axis.ticks = element_line(size=1),
              legend.text=element_text(size=20)))
}

## Rename the clusters

In [ ]:
tcells@meta.data$ct_highres <- 'FALSE'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '2'] <- 'Tna_1'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '4'] <- 'Tna_2'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '13'] <- 'Tna_3'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '5'] <- 'Tna_4'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '3'] <- 'Tna_5'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '16'] <- 'Tna_6'

tcells@meta.data$ct_highres[tcells@meta.data$recluster == '20'] <- 'IFN_sign'

tcells@meta.data$ct_highres[tcells@meta.data$recluster == '0'] <- 'TCM_1'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '12'] <- 'TCM_2'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '6'] <- 'TCM_3'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '8'] <- 'TCM_4'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '1'] <- 'TCM_5'

tcells@meta.data$ct_highres[tcells@meta.data$recluster == '21'] <- 'TEM_1'
tcells@meta.data$ct_highres[tcells@meta.data$recluster == '22'] <- 'TEM_2'

tcells@meta.data$ct_highres[tcells@meta.data$recluster == '18'] <- 'Treg'

In [ ]:
Idents(tcells) <- 'ct_highres'
levels(tcells) <- c('Tna_1', 'Tna_2', 'Tna_3', 'Tna_4', 'Tna_5', 'Tna_6', 'IFN_sign', 'TCM_1', 'TCM_2', 'TCM_3', 'TCM_4', 'TCM_5', 'TEM_1', 'TEM_2', 'Treg')

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 11)
DimPlot(tcells, reduction = "umap", label = TRUE, label.size = 8, repel = TRUE, pt.size = 0.7) + 
theme(axis.line = element_line(size=1),
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      axis.ticks = element_line(size=1),
      legend.text=element_text(size=20))
options(repr.plot.width = 11, repr.plot.height = 11)


## Add diagnosis and twin pair information

In [ ]:
#add diagnosis and twin pair information 
diagnosis <- read.csv(file = 'diagnosis.csv')
tcells$diagnosis <- 'FALSE'
tcells$twin_pair <- 'FALSE'
for(i in 1:nrow(tcells@meta.data)){
    tcells@meta.data$diagnosis[i] <- filter(diagnosis, sample == tcells@meta.data$sample[i])$diagnosis
    tcells@meta.data$twin_pair[i] <- filter(diagnosis, sample == tcells@meta.data$sample[i])$twin_pair
}

# Functions for working with the ready T cells file

In [ ]:
#plotting the UMAP
cols <- c("#FFD258", "#E8BF4D", "#C09725", "#D89510", "#CBA049", "#EEBC4A", "#A2AC13", "#557F7A", "#409088", "#387C75", "#3B5B58", "#226861", "#7AA591", "#CBD49C", "#FEEDC3")
umap_mapping <- DimPlot(tcells, reduction = 'umap', label = TRUE, pt.size = 0.7, label.size = 5, repel = TRUE, cols = cols) + theme(legend.position = "none") +
      geom_density2d(data=
      data.frame(Embeddings(subset(tcells, idents = c('Tna_4', 'Tna_5', 'TCM_4'))[["umap"]])),
      aes(x = UMAP_1, y = UMAP_2), col="red", bins = 8)
umap_mapping
ggsave(umap_mapping, file ="output/umap_mapping_selected.png")
ggsave(umap_mapping, file ="output/umap_mapping_selected.pdf")

In [ ]:
#plotting the feature plots
markers <- c('CD45RA-Ab', 'CD45RO-Ab', 'FOXP3', 'CX3CR1')
umap_overlay <- FeaturePlot(tcells, reduction = 'umap', label = F, features = markers, pt.size = 0.01, cols = color_grad_flow2, ncol = 2) +
                plot_annotation(title = 'T cells markers') & theme_few() &
        theme(axis.title.x=element_blank(),
                axis.text.x=element_blank(),
                axis.ticks.x=element_blank(),
                axis.title.y=element_blank(),
                axis.text.y=element_blank(),
                axis.ticks.y=element_blank(),
                legend.position = "none")
ggsave(umap_overlay, file="output/umap_overlay_tcells_markers.pdf", width = 6, height = 6)

In [ ]:
# DE for AB assay with LR on twin pairs and all clusters
diff_markers_ab  <- function(data, cell_type){
        diff  <- subset(data, subset = (twin_pair != "none" & ct_highres == cell_type)) %>%
                        FindMarkers(., test.use="LR", latent.vars = "twin_pair", group.by = "diagnosis", ident.1 = "MS", ident.2 = "Healthy", logfc.threshold = 0, features = rownames(data), assay = "Ab", min.pct = 0.05) %>%
                        mutate(cell.type = cell_type, gene = rownames(.))
    }


diff_markers_ab_res  <- unique(levels(tcells)) %>% map_dfr(~ diff_markers_ab(data = tcells, cell_type = .))

# plot the selected AB for all clusters
diff_AB_Name_ab  <- diff_markers_ab_res %>% subset(gene == "AB_Name")
color_qual_flow2 <- c("TRUE" = "#CC242A", up = "#CC242A", "FALSE" = "#557F7A", down = "#557F7A")
reverselog_trans <- function(base = exp(1)) {
    trans <- function(x) -log(x, base)
    inv <- function(x) base^(-x)
    trans_new(paste0("reverselog-", format(base)), trans, inv,
              log_breaks(base = base),
              domain = c(1e-100, Inf))
    }
diff_AB_Name_ab  <- diff_AB_Name_ab[order(diff_AB_Name_ab$p_val_adj, decreasing = T),]
diff_AB_Name_ab$cell.type  <- factor(diff_AB_Name_ab$cell.type, levels = unique(diff_AB_Name_ab$cell.type))
diff_AB_Name_ab_up  <- subset(diff_AB_Name_ab, avg_log2FC > 0)
AB_Name_balloon_path <- ggplot(diff_AB_Name_ab_up, aes(y=cell.type, x= p_val_adj, fill = avg_log2FC > 0))+
        scale_fill_manual(values = color_qual_flow2)+
        geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
        geom_segment( aes(yend=cell.type, xend=1), col= "black") +
        geom_point(shape=21, aes(size = abs(avg_log2FC))) +
        scale_x_continuous(trans=reverselog_trans(10))+
        scale_size_continuous(range = c(2, 7)) +
        theme_tufte()+ xlab("p value (adj)") + ylab("") +
        theme(text=element_text(family="Helvetica"),
          axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right")
AB_Name_balloon_path
ggsave(AB_Name_balloon_path, file="output/AB_Name_lolli.pdf", width = 6.5, height = 4)

In [ ]:
#Run the DGE on the selected clusters with building Volcano plots
object <- tcells
DefaultAssay(object) <- 'RNA'
featurestcells <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC", x = rownames(object), value = TRUE)
featurestcells <- featurestcells[!(featurestcells%in%markers.remove)]

# DEGs for all clusters
diff_sig_markers  <- function(data, cell_type){
        sig_diff  <- subset(data, subset = (twin_pair != "none" & cell.type == cell_type)) %>% 
                        FindMarkers(., test.use="LR", latent.vars = "twin_pair", group.by = "diagnosis", ident.1 = "MS", ident.2 = "Healthy", logfc.threshold = 0.05, min.pct = 0.1, features = featurestcells) %>% 
                        mutate(cell.type = cell_type, gene = rownames(.))
    }
cell_types <- c('Tna_4', 'Tna_5', 'TCM_4') #here the selected clusters
sig_markers  <- cell_types %>% map_dfr(~ diff_sig_markers(data = object, cell_type = .))
sig_markers
# plot volcanos per cluster
plot_volcano  <- function(data, cell_type){
    volcano <- ggplot(subset(data, cell.type == cell_type), aes(x = avg_log2FC, y = -log10(p_val_adj))) +
    geom_vline(xintercept = 0) +
    geom_hline(yintercept = -log10(0.05), color ="grey", linetype ="dashed") +
    geom_point(data = subset(data, cell.type == cell_type),
                color = "grey", alpha = 0.5) +
    geom_point(data = subset(data, cell.type == cell_type & avg_log2FC > 0 & p_val_adj < .05)[1:20,],
                fill = "#CC242A", alpha = 1, shape=21, size= 2.5) +
    geom_point(data = subset(data, cell.type == cell_type & avg_log2FC < 0 & p_val_adj < .05)[1:20,],
                fill = "#557F7A", alpha = 1, shape=21, size= 2.5) +
    geom_text_repel(data=subset(data, cell.type == cell_type & p_val_adj < .05)[1:20,], aes(label = gene))+
    theme_linedraw() +
    theme(panel.grid = element_blank(), legend.position = "none") +
    xlab("log2(average fold change)") +
    ylab("-log10(p-value)")
    volcano
    ggsave(volcano, file=paste0("DE_tcells/volcano_", cell_type, ".pdf"), height = 4, width = 5)
    }

cell_types %>% map(~ plot_volcano(data = sig_markers, cell_type = .))

## Explore the expansion

In [ ]:
#create a sample_clono column
tcells@meta.data$sample_clono <- paste(tcells@meta.data$sample.effect, tcells@meta.data$TCR_clonotype_id, sep = '_')

#crea expanded column
tcells@meta.data$TCR_frequency <- as.numeric(tcells@meta.data$TCR_frequency)
tcells@meta.data$expand <- 'FALSE'
tcells@meta.data$expand[tcells@meta.data$TCR_frequency > 2] <- 'expanded'
tcells@meta.data$expand[tcells@meta.data$TCR_frequency < 3] <- 'non-expanded'
unique(tcells@meta.data$expand)

options(repr.plot.width = 16, repr.plot.height = 11)
expanded_cells <- DimPlot(tcells, reduction = "umap", label = TRUE, label.size = 8, cells.highlight = rownames(filter(tcells@meta.data, expand == 'expanded')), sizes.highlight = 0.6, repel = TRUE, pt.size = 0.8, cols.highlight = '#FF7B65') + 
theme(text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))
expanded_cells
ggsave(expanded_cells, file = 'expansion/umap_expandedcells.pdf', width = 16, height = 11)
options(repr.plot.width = 11, repr.plot.height = 11)


In [ ]:
#exlore the expansion per cluster/sample/diagnosis
diagnosis_list <- c('Healthy', 'MS')
clusters_list <- levels(object)
alldiagnosis_expansion_together <- c()
for (d in 1:length(diagnosis_list)){
    persample_percluster <- data.frame(matrix(NA, nrow = length(unique(filter(object@meta.data, diagnosis == diagnosis_list[d])$twin)), ncol = length(clusters_list)))
    colnames(persample_percluster) <- clusters_list
    rownames(persample_percluster) <- unique(filter(object@meta.data, diagnosis == diagnosis_list[d])$twin)
    for(i in 1:nrow(persample_percluster)){
        for (c in 1:ncol(persample_percluster)){
            persample_percluster[i, c] <- 100 * nrow(filter(object@meta.data, ct_highres == colnames(persample_percluster)[c] & expand == 'expanded' & twin == rownames(persample_percluster)[i])) /
                                            nrow(filter(object@meta.data, ct_highres == colnames(persample_percluster)[c] & twin == rownames(persample_percluster)[i]))
            if(is.na(persample_percluster[i, c])){persample_percluster[i, c] <- 0}
        }
    }
    persample_percluster$diagnosis <- diagnosis_list[d]
    alldiagnosis_expansion_together <- rbind(alldiagnosis_expansion_together, persample_percluster)
}
alldiagnosis_expansion_together$twin <- rownames(alldiagnosis_expansion_together)
alldiagnosis_expansion_together
write.csv(alldiagnosis_expansion_together, file = './expansion/alldiagnosis_together_persample_percluster.csv')

## Build up the pseudotime trajectory

In [ ]:
Idents(tcells) <- 'ct_highres'
levels(tcells) <- c('Tna_1', 'Tna_2', 'Tna_3', 'Tna_4', 'Tna_5', 'Tna_6', 'IFN_sign', 'TCM_1', 'TCM_2', 'TCM_3', 'TCM_4', 'TCM_5', 'TEM_1', 'TEM_2', 'Treg')
object <- tcells

#monocle v3:
gene_annotation <- as.data.frame(rownames(object@reductions[["pca"]]@feature.loadings),
                                 row.names = rownames(object@reductions[["pca"]]@feature.loadings))
colnames(gene_annotation) <- "gene_short_name"


cell_metadata <- as.data.frame(object@assays[["RNA"]]@counts@Dimnames[[2]],
                               row.names = object@assays[["RNA"]]@counts@Dimnames[[2]])
colnames(cell_metadata) <- "barcode"


New_matrix <- object@assays[["RNA"]]@counts
New_matrix <- New_matrix[rownames(object@reductions[["pca"]]@feature.loadings), ]
expression_matrix <- New_matrix

library(monocle3)

cds_from_seurat <- new_cell_data_set(expression_matrix,
                                     cell_metadata = cell_metadata,
                                     gene_metadata = gene_annotation)


recreate.partition <- c(rep(1, length(cds_from_seurat@colData@rownames)))
names(recreate.partition) <- cds_from_seurat@colData@rownames
recreate.partition <- as.factor(recreate.partition)

cds_from_seurat@clusters@listData[["UMAP"]][["partitions"]] <- recreate.partition



list_cluster <- object@active.ident
names(list_cluster) <- object@assays[["RNA"]]@data@Dimnames[[2]]

cds_from_seurat@clusters@listData[["UMAP"]][["clusters"]] <- list_cluster



cds_from_seurat@clusters@listData[["UMAP"]][["louvain_res"]] <- "NA"


cds_from_seurat@int_colData@listData$reducedDims@listData[["UMAP"]] <-object@reductions[["umap"]]@cell.embeddings


cds_from_seurat@preprocess_aux$gene_loadings <- object@reductions[["pca"]]@feature.loadings
cds_from_seurat <- learn_graph(cds_from_seurat, use_partition = F)

plot_cells(cds_from_seurat, 
                   color_cells_by = 'cluster',
           
                   label_groups_by_cluster=TRUE,
                   label_leaves=T,
                   label_branch_points=TRUE,
           graph_label_size=4)

cds_from_seurat <- order_cells(cds_from_seurat, root_cells = WhichCells(object, idents = 'Tna_1'))

In [ ]:
#add the data from the monocle pseudotime analysis into the seurat object
object <- AddMetaData(
  object = object,
  metadata = cds_from_seurat@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "Pseudotime"
)

In [ ]:
options(repr.plot.width=15, repr.plot.height=13)

In [ ]:
options(repr.plot.width=15, repr.plot.height=13)
color_grad_flow2 <- c("#331820", "#4D4F55", "#55626B", "#5A767E", "#628B8A", "#709B91", "#82AA96", "#98B89A", "#B0C6A2", "#C9D3AB", "#E4E0B6", "#FEEDC3")
pseudotime_umap <- FeaturePlot(object, features = 'Pseudotime', pt.size = 0.7, label = TRUE, label.size = 6, repel = TRUE, cols = c("#FEEDC3", "#557F7A"))
print(pseudotime_umap)
ggsave(pseudotime_umap, file="output/pseudotime_umap.pdf")
ggsave(pseudotime_umap, file="output/pseudotime_umap.png")
options(repr.plot.width=11, repr.plot.height=11)

In [ ]:
# plot pseudotime vs protein markers
data_ab <- t(tcells@assays$Ab@data)
meta_monocle <- tcells@meta.data
data_ab_pseudo <- data.frame(cell_id = names(meta_monocle), pseudotime = meta_monocle$Pseudotime, cluster = tcells[["ct_highres"]]$ct_highres, data_ab)

cluster_pseudotime_stats <- data_ab_pseudo %>% group_by(cluster) %>% summarize_at("pseudotime", c(mean, sd))
colnames(cluster_pseudotime_stats)[2:3] <- c("mean", "sd")

cluster_pseudotime_stats$mean_minus_sd <- (cluster_pseudotime_stats$mean-cluster_pseudotime_stats$sd)
cluster_pseudotime_stats$mean_plus_sd <- (cluster_pseudotime_stats$mean+cluster_pseudotime_stats$sd)

cluster_pseudotime_stats_m <- cluster_pseudotime_stats[,c(1, 2, 4, 5)] %>% melt


pseudo_abs <- ggplot(subset(data_ab_pseudo, cluster != "Treg"), aes(x = pseudotime, y = CD25.Ab)) + 
geom_vline(xintercept = mean(subset(data_ab_pseudo, cluster %in% c("Tna_4"))$pseudotime), linetype = "dashed", color = "grey") +
geom_vline(xintercept = mean(subset(data_ab_pseudo, cluster %in% c("Tna_5"))$pseudotime), linetype = "dashed", color = "grey") +
geom_rug(data = subset(cluster_pseudotime_stats_m, cluster == "Tna_4" & variable == "mean"), col = "#CB181E", sides = "b", inherit.aes = FALSE, size = 2, aes(x = value), length = unit(0.07, "npc")) + 
geom_rug(data = subset(cluster_pseudotime_stats_m, cluster == "Tna_5" & variable == "mean"), col = "#CB181E", sides = "b", inherit.aes = FALSE, size = 2, aes(x = value), length = unit(0.07, "npc")) + 
coord_cartesian(ylim=c(0,2.5)) +
geom_smooth(fill = "#FFD443", col = "#FFD443", aes(x = pseudotime, y = CD25.Ab), method = "gam", formula = y ~ s(x, bs = "cs", fx = TRUE, k = 6)) +
geom_smooth(fill = "#4D7C76", col = "#4D7C76", aes(x = pseudotime, y = CD45RA.Ab), method = "gam", formula = y ~ s(x, bs = "cs", fx = TRUE, k = 6)) +
geom_smooth(fill = "#D2D699", col = "#D2D699", aes(x = pseudotime, y = CD45RO.Ab), method = "gam", formula = y ~ s(x, bs = "cs", fx = TRUE, k = 6)) +
geom_smooth(fill = "#7CA38B", col = "#7CA38B", aes(x = pseudotime, y = CCR7.Ab), method = "gam", formula = y ~ s(x, bs = "cs", fx = TRUE, k = 6)) +
scale_color_manual(values = cols_clust) +
theme_few()
pseudo_abs
ggsave(pseudo_abs, file = "output/pseudotime_abs_Tna4_mapping_monocle.pdf", width = 5, height = 3.5)


In [ ]:
#save the object for diffusion map building in scanpy
library(SeuratDisk)
DefaultAssay(object) = "integrated"
SaveH5Seurat(object, filename = "./tcells_only_monocle.h5Seurat", overwrite = T)
Convert("tcells_only_monocle.h5Seurat", dest = "h5ad")

## Build up diffusion maps in scanpy (with Python!)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import scipy as sp
import random
import scvelo as scv
import numpy as np
import umap
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, frameon=False, figsize=(7, 7), facecolor='white')
from platform import python_version
print(python_version())
import PyQt5
%matplotlib qt

In [ ]:
#load the object, perform neighbouring and dimensional reduction
random.seed(0)
adata = sc.read_h5ad("./tcells_only_slingshot.h5ad")
sc.pp.highly_variable_genes(adata, n_top_genes = 1000, flavor="seurat")
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=11)
sc.tl.diffmap(adata)

palette = ["#FFD258", "#E8BF4D", "#C09725", "#D89510", "#CBA049", "#EEBC4A", "#A2AC13", "#557F7A", "#409088", "#387C75", "#3B5B58", "#226861", "#7AA591", "#CBD49C", "#FEEDC3"]

#reorder the categories
adata.obs['ct_highres'].cat.reorder_categories(['Tna_1', 'Tna_2', 'Tna_3', 'Tna_4', 'Tna_5', 'Tna_6', 'IFN_sign', 'TCM_1', 'TCM_2', 'TCM_3', 'TCM_4', 'TCM_5', 'TEM_1', 'TEM_2', 'Treg'], inplace=True)

#plot the diffusion maps
%matplotlib inline
sc.set_figure_params(dpi=300, frameon=False, figsize=(10, 10), facecolor='white', vector_friendly = True)
sc.pl.diffmap(adata,components=[1,2],color=["ct_highres"], frameon=True, projection = "2d",
              legend_loc="right margin", palette = palette, legend_fontsize = 12, legend_fontweight = 'normal', size = 20,
             save = '_monocle_withlegend.pdf')

sc.set_figure_params(dpi=300, frameon=False, figsize=(10, 10), facecolor='white', vector_friendly = True)
sc.pl.diffmap(adata,components=[1,2],color=["ct_highres"], frameon=True, projection = "2d",
              legend_loc="on data", palette = palette, legend_fontsize = 12, legend_fontweight = 'normal', size = 20,
             save = '_monocle.pdf')

sc.set_figure_params(dpi=300, frameon=False, figsize=(9, 10), facecolor='white', vector_friendly = True)
sc.pl.diffmap(adata,components=[1,2],color=["ct_highres"], frameon=True, projection = "2d",
              legend_loc="none", palette = palette, legend_fontsize = 12, legend_fontweight = 'normal', size = 20,
             save = '_monocle_without_legend.pdf')

## Run Dorothea on T cells data

In [ ]:
# transcription factor activity
library(viper)
library(dorothea)

dorothea_regulon_human <- get(data("dorothea_hs", package = "dorothea"))

regulon <- dorothea_regulon_human %>%
filter(confidence %in% c("A","B","C"))

Th_naive <- tcells %>% subset(subset = (ct_highres %in% c("Tna_1", "Tna_2", "Tna_3", "Tna_4", "Tna_5", "Tna_6", "IFN_sign") & twin_pair != "none") )

Th_naive <- run_viper(Th_naive, regulon,
options = list(method = "scale", minsize = 4, 
eset.filter = FALSE, cores = 32, 
verbose = TRUE))


DefaultAssay(object = Th_naive) <- "dorothea"
# phagocytes <- ScaleData(phagocytes)

diff_markers_TF <- function(data, cell_type){
diff <- subset(data, subset = (twin_pair != "none" & ct_highres == cell_type)) %>% 
FindMarkers(., test.use="LR", latent.vars = "twin_pair", group.by = "diagnosis", ident.1 = "MS", ident.2 = "Healthy", logfc.threshold = 0) %>%
mutate(cell.type = cell_type, gene = rownames(.))
}

TF_diff_res <- unique(Idents(Th_naive)) %>% map_dfr(~ diff_markers_TF(data = Th_naive, cell_type = .))
TF_sig_markers <- TF_diff_res %>% subset(p_val_adj <.05)
TF_sig_markers %>% View
write.csv(TF_sig_markers, file ="output/DETF_activity_naive.csv")

TF_violins <- Th_naive %>% subset(subset = (twin_pair != "none" & ct_highres == "Tna_4")) %>% 
VlnPlot(features = "RELA", split.by = "diagnosis", group.by = "twin_pair",
pt.size = 1, combine = FALSE, assay = "dorothea")

TF_violins

data_exp <- Th_naive %>% subset(subset = (twin_pair != "none" & ct_highres == "Tna_5")) %>% 
AverageExpression(assay ="dorothea", features = c("STAT4", "RELA"), group.by = "sample") %>%
melt %>% rename(gene = Var1, sample = Var2, expression = value) %>% merge(md) %>%
arrange(gene, twin_pair, diagnosis) 

normalize <- function(x){(x- min(x))/(max(x)-min(x))}

mat <- subset(data_exp, gene == "STAT4")$expression %>% normalize %>%
matrix(ncol = (2), byrow = T, dimnames = list(paste("twin pair", unique(data_exp$twin_pair))))

white.black2 <- colorRampPalette(c("white", "black"))(n = 20)


pheatmap(mat = mat, 
scale= "none",
color = white.black2,
# breaks = breaks,
cluster_rows=F,
cluster_cols=F,
# annotation_row = annotation,
# annotation_colors = ann_col,
cellwidth = 10, cellheight = 10,
border_color = NA,
filename= "output/TF_STAT4_Tna5_heatmap_norm.pdf"
)

# Subset P8 samples, recluster it, remove everything except of monos

In [ ]:
Idents(pbmc) <- 'sort'
## change "sort" value for sample X139 (p5 and p8 were processed in this sample together):
pbmc@meta.data$sort[pbmc@meta.data$sample == 'X139'] <- 'p8'
monos <- subset(pbmc, idents = 'p8')
monos

In [ ]:
## change back "sort" value for sample X139:
pbmc@meta.data$sort[pbmc@meta.data$sample == 'X139'] <- 'p5'

In [ ]:
DefaultAssay(monos) <- 'RNA'

In [ ]:
monos <- FindVariableFeatures(monos, selection.method = "vst", nfeatures = 2000)
length(x = VariableFeatures(object = monos))

In [ ]:
monos <- ScaleData(monos, features = VariableFeatures(object = monos), vars.to.regress = c("nCount_RNA", "percent.mito"))
monos <- RunPCA(monos, features = VariableFeatures(object = monos))

In [ ]:
ElbowPlot(object = monos, ndims = 50)

In [ ]:
for (i in 1:25){
    print(DimHeatmap(object = monos, dims = i, cells = 500, balanced = TRUE))
}

## Cluster the cells

In [ ]:
monos <- FindNeighbors(object = monos, dims = 1:14)

In [ ]:
monos <- FindClusters(object = monos, resolution = 0.5)

## Run UMAP

In [ ]:
monos <- RunUMAP(monos, dims = 1:14)

In [ ]:
#vst, umap, dims 14, res. 0.5
DimPlot(monos, reduction = 'umap', label = TRUE, label.size = 8)
DimPlot(monos, reduction = 'umap', label = TRUE, group.by = 'sample')

In [ ]:
#dims 14, 0.5
VlnPlot(monos, features = 'nFeature_RNA', pt.size = 0.01)

## Cluster markers

In [ ]:
DefaultAssay(monos) <- 'RNA'

In [ ]:
featuresmonos <- rownames(monos)
monos.markers1 <- FindAllMarkers(object = monos, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featuresmonos)

In [ ]:
#sort top 30 first markers per cluster
monos.markers1_sorted <- c()
for (i in 1:length(levels(monos.markers1$cluster))){
    monos.markers1_level <- filter(monos.markers1, cluster == levels(monos.markers1$cluster)[i])
    monos.markers1_level <- monos.markers1_level[order(-monos.markers1_level$avg_log2FC), ]
    monos.markers1_level <- monos.markers1_level[1:30, ]
    monos.markers1_level <- monos.markers1_level[!is.na(monos.markers1_level$avg_log2FC), ]
    monos.markers1_sorted <- rbind(monos.markers1_sorted, monos.markers1_level)
    }
monos.markers1_sorted_top30 <- monos.markers1_sorted
write.csv(monos.markers1_sorted_top30, file = './monos_top30_first.csv')

In [ ]:
#compute the average expression for heatmaps
cluster.averages_monos <- AverageExpression(monos, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
#sort top5 first markers per cluster
monos.markers1_sorted <- c()
for (i in 1:length(levels(monos.markers1$cluster))){
    monos.markers1_level <- filter(monos.markers1, cluster == levels(monos.markers1$cluster)[i])
    monos.markers1_level <- monos.markers1_level[order(-monos.markers1_level$avg_log2FC), ]
    monos.markers1_level <- monos.markers1_level[1:5, ]
    monos.markers1_level <- monos.markers1_level[!is.na(monos.markers1_level$avg_log2FC), ]
    monos.markers1_sorted <- rbind(monos.markers1_sorted, monos.markers1_level)
    }
monos.markers1_sorted_top5 <- monos.markers1_sorted

In [ ]:
#plot top5 markers per cluster in heatmap
DoHeatmap(cluster.averages_monos, features = monos.markers1_sorted_top5$gene)+ theme(text = element_text(size = 20, face = "bold"))
#plot the nfeatures per cell in VlnPlot (helpuf for sorting out low quality cells)
VlnPlot(monos, features = 'nFeature_RNA', pt.size = 0.01)

## Subset only the Monos, DC and B/DC mix

In [ ]:
monos@meta.data$old_clusters <- monos@meta.data$seurat_clusters
monos <- subset(monos, idents = c(3,4,9,12,13,15,16))
monos

In [ ]:
monos <- FindVariableFeatures(monos, selection.method = "vst", nfeatures = 2000)
length(x = VariableFeatures(object = monos))

In [ ]:
monos <- ScaleData(monos, features = VariableFeatures(object = monos), vars.to.regress = c("nCount_RNA", "percent.mito"))
monos <- RunPCA(monos, features = VariableFeatures(object = monos))

In [ ]:
# ProjectDim scores each feature in the dataset (including features not included in the PCA) based on their correlation 
# with the calculated components. Though we don't use this further here, it can be used to identify markers that 
# are strongly correlated with cellular heterogeneity, but may not have passed through variable feature selection. 
# The results of the projected PCA can be explored by setting `projected = TRUE`in the functions above
monos <- ProjectDim(object = monos)

In [ ]:
ElbowPlot(object = monos, ndims = 50)

In [ ]:
for (i in 1:25){
    print(DimHeatmap(object = monos, dims = i, cells = 500, balanced = TRUE))
}

## Cluster the cells

In [ ]:
monos <- FindNeighbors(object = monos, dims = 1:8)

In [ ]:
monos <- FindClusters(object = monos, resolution = 0.5)

## Run UMAP after reclustering

In [ ]:
monos <- RunUMAP(monos, dims = 1:8)

In [ ]:
DimPlot(monos, reduction = 'umap', label = TRUE, label.size = 8)
DimPlot(monos, reduction = 'umap', label = TRUE, group.by = 'sample')
VlnPlot(monos, features = 'nFeature_RNA', pt.size = 0.01)

## Cluster markers (second run)

In [ ]:
DefaultAssay(monos) <- 'RNA'

In [ ]:
featuresmonos <- rownames(monos)
monos.markers1 <- FindAllMarkers(object = monos, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featuresmonos)

In [ ]:
#sort top 30 first markers per cluster
monos.markers1_sorted <- c()
for (i in 1:length(levels(monos.markers1$cluster))){
    monos.markers1_level <- filter(monos.markers1, cluster == levels(monos.markers1$cluster)[i])
    monos.markers1_level <- monos.markers1_level[order(-monos.markers1_level$avg_log2FC), ]
    monos.markers1_level <- monos.markers1_level[1:30, ]
    monos.markers1_level <- monos.markers1_level[!is.na(monos.markers1_level$avg_log2FC), ]
    monos.markers1_sorted <- rbind(monos.markers1_sorted, monos.markers1_level)
    }
monos.markers1_sorted_top30 <- monos.markers1_sorted
write.csv(monos.markers1_sorted_top30, file = './monos_top30_second.csv')

In [ ]:
#compute the average expression for heatmaps
cluster.averages_monos <- AverageExpression(monos, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
#sort top5 first markers per cluster
monos.markers1_sorted <- c()
for (i in 1:length(levels(monos.markers1$cluster))){
    monos.markers1_level <- filter(monos.markers1, cluster == levels(monos.markers1$cluster)[i])
    monos.markers1_level <- monos.markers1_level[order(-monos.markers1_level$avg_log2FC), ]
    monos.markers1_level <- monos.markers1_level[1:5, ]
    monos.markers1_level <- monos.markers1_level[!is.na(monos.markers1_level$avg_log2FC), ]
    monos.markers1_sorted <- rbind(monos.markers1_sorted, monos.markers1_level)
    }
monos.markers1_sorted_top5 <- monos.markers1_sorted

In [ ]:
#plot top5 markers per cluster in heatmap
DoHeatmap(cluster.averages_monos, features = monos.markers1_sorted_top5$gene)+ theme(text = element_text(size = 20, face = "bold"))
#plot the nfeatures per cell in VlnPlot (helpuf for sorting out low quality cells)
VlnPlot(monos, features = 'nFeature_RNA', pt.size = 0.01)

## Remove doublets, perform integration due to detected batch effect

In [ ]:
#remove cluster 4, 5, 12, 8, 9
monos <- subset(monos, idents = c(4,5,12,8,9), invert = TRUE)
monos

In [ ]:
monos.list <- SplitObject(monos, split.by = "sample.effect")
monos.list

In [ ]:
monos.list <- lapply(X = monos.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 5000)
})

for (i in 1:length(monos.list)){
     VariableFeatures(monos.list[[i]])
}

In [ ]:
features <- SelectIntegrationFeatures(object.list = monos.list, nfeatures = 5000)

In [ ]:
monos.list <- lapply(X = monos.list, FUN = function(x) {
    x <- ScaleData(x, features = features, verbose = FALSE)
    x <- RunPCA(x, features = features, verbose = FALSE)
})

In [ ]:
monos.anchors <- FindIntegrationAnchors(object.list = monos.list, anchor.features = features, reduction = "rpca")

In [ ]:
monos.integrated <- IntegrateData(anchorset = monos.anchors, k.weight = 50)

## working with the integrated file

In [ ]:
DefaultAssay(monos.integrated) <- 'integrated'

In [ ]:
monos.integrated <- FindVariableFeatures(monos.integrated, selection.method = "vst", nfeatures = 1500)

In [ ]:
monos.integrated <- ScaleData(monos.integrated, features = VariableFeatures(object = monos.integrated), vars.to.regress = c("nCount_RNA", "percent.mito"))
monos.integrated <- RunPCA(monos.integrated, features = VariableFeatures(object = monos.integrated))

In [ ]:
ElbowPlot(object = monos.integrated, ndims = 50)

In [ ]:
for (i in 1:25){
    print(DimHeatmap(object = monos.integrated, dims = i, cells = 500, balanced = TRUE))
}

In [ ]:
monos.integrated <- FindNeighbors(object = monos.integrated, dims = 1:12)

In [ ]:
monos.integrated <- FindClusters(object = monos.integrated, resolution = 0.4)

In [ ]:
monos.integrated <- RunUMAP(monos.integrated, dims = 1:12)

In [ ]:
DimPlot(monos.integrated, reduction = 'umap', label = TRUE, label.size = 8)
DimPlot(monos.integrated, reduction = 'umap', label = TRUE, group.by = 'sample')

In [ ]:
VlnPlot(monos.integrated, features = 'nFeature_RNA', pt.size = 0.01)

## Cluster markers of the integrated file

In [ ]:
DefaultAssay(monos.integrated) <- 'RNA'

In [ ]:
monos.integrated.markers1 <- FindAllMarkers(object = monos.integrated, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = rownames(monos.integrated))

In [ ]:
#sort markers
monos.integrated.markers1_sorted <- c()
for (i in 1:length(levels(monos.integrated.markers1$cluster))){
    monos.integrated.markers1_level <- filter(monos.integrated.markers1, cluster == levels(monos.integrated.markers1$cluster)[i])
    monos.integrated.markers1_level <- monos.integrated.markers1_level[order(-monos.integrated.markers1_level$avg_log2FC), ]
    monos.integrated.markers1_level <- monos.integrated.markers1_level[1:30, ]
    monos.integrated.markers1_level <- monos.integrated.markers1_level[!is.na(monos.integrated.markers1_level$avg_log2FC), ]
    monos.integrated.markers1_sorted <- rbind(monos.integrated.markers1_sorted, monos.integrated.markers1_level)
    }
monos.integrated.markers1_sorted_top30 <- monos.integrated.markers1_sorted
monos.integrated.markers1_sorted_top30
write.csv(monos.integrated.markers1_sorted_top30, file = './monos.integrated_top30.csv')

In [ ]:
cluster.averages_monos.integrated <- AverageExpression(monos.integrated, assay = "RNA", return.seurat = TRUE) # , verbose = FALSE)

In [ ]:
monos.integrated.markers1_sorted <- c()
for (i in 1:length(levels(monos.integrated.markers1$cluster))){
    monos.integrated.markers1_level <- filter(monos.integrated.markers1, cluster == levels(monos.integrated.markers1$cluster)[i])
    monos.integrated.markers1_level <- monos.integrated.markers1_level[order(-monos.integrated.markers1_level$avg_log2FC), ]
    monos.integrated.markers1_level <- monos.integrated.markers1_level[1:5, ]
    monos.integrated.markers1_level <- monos.integrated.markers1_level[!is.na(monos.integrated.markers1_level$avg_log2FC), ]
    monos.integrated.markers1_sorted <- rbind(monos.integrated.markers1_sorted, monos.integrated.markers1_level)
    }
monos.integrated.markers1_sorted_top5 <- monos.integrated.markers1_sorted
monos.integrated.markers1_sorted_top5

In [ ]:
DoHeatmap(cluster.averages_monos.integrated, features = monos.integrated.markers1_sorted_top5$gene)+ theme(text = element_text(size = 20, face = "bold"))

## Rename clusters, find markers and plot

In [ ]:
phagocytes <- monos.integrated

In [ ]:
# create md file 
md  <- phagocytes@meta.data[,c("sample", "twin_pair", "diagnosis")] %>% unique
# define features
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC", x = rownames(phagocytes), value = TRUE)
features <- rownames(phagocytes)[!(rownames(phagocytes) %in% markers.remove)]
markers_ab  <- rownames(phagocytes@assays$Ab)

# rename the idents
phagocytes  <- RenameIdents(phagocytes, "0" = "Non-classical monocytes", "9" = "Non-classical monocytes",
        "5" = "Intermediate monocytes", "6" = "Intermediate monocytes", "1" = "Classical monocytes", "4" = "Classical monocytes", 
                "2" = "Classical monocytes", "3" = "Dendritic cells", "8" = "Plasmacytoid dendritic cells", "7" = "Megakaryocytes")


In [ ]:
#find markers
phagocyte.markers <- FindAllMarkers(phagocytes, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
top_phagocyte_markers  <- phagocyte.markers %>%
    group_by(cluster) %>%
    slice_max(n = 10, order_by = avg_log2FC)

selected_phagocyte_markers  <- c("FCGR3A", "LY6E", "S100A8", "VCAN", "CD14", "FCER1A", "HLA-DQA1", "CLEC10A", "CLEC9A", "IRF8", "GZMB", "PPBP", "GNG11")    

#create dotplot
dotplot_phagocytes  <- DotPlot(phagocytes, features = selected_phagocyte_markers, cols = c("white", "#557F7A"), cluster.idents = T) + RotatedAxis()
dotplot_phagocytes
ggsave(dotplot_phagocytes, file = "output/dotplot_phagocytes_signature_genes.pdf", height = 3, width = 8)

cols_phagocytes  <- c("Classical monocytes" = "#557F7A", "Intermediate monocytes" = "#7AA591", "Non-classical monocytes" = "#CBD49C", "Dendritic cells" = "#FFD258", "Plasmacytoid dendritic cells" = "grey", "Megakaryocytes" = "#FEEDC3", "CD14dim" = "#CC242A")

#create umap
umap_clust  <- DimPlot(phagocytes, reduction = 'umap', label = FALSE, pt.size = 1, cols = cols_phagocytes) + theme(legend.position = "none")
ggsave(umap_clust, file = "output/umap_clust.png")
phagocytes@meta.data$cell.annotation <- Idents(phagocytes) 

## subclustering of classical monocytes based on epitope expression to obtain CD14 dim population
code based on: https://broadinstitute.github.io/2020_scWorkshop/cite-seq.html#add-the-protein-expression-levels-to-the-seurat-object


In [ ]:
# select classical monocyte subcluster
DimPlot(phagocytes, reduction = 'umap', label = TRUE, pt.size = 1, group.by = "integrated_snn_res.0.8")
Idents(phagocytes)  <- phagocytes@meta.data$integrated_snn_res.0.8

cl_mono  <- subset(phagocytes, idents = "7")

# cluster based on Ab expression
DefaultAssay(cl_mono) <- "Ab"
cl_mono <- ScaleData(cl_mono)
cl_mono <- RunPCA(cl_mono, features = rownames(cl_mono), reduction.name = "pca_Ab", reduction.key = "pca_Ab_", verbose = FALSE)

Ab.data <- GetAssayData(cl_mono, slot = "data")
Ab.dist <- dist(t(Ab.data))

cl_mono[["tsne_Ab"]] <- RunTSNE(Ab.dist, assay = "Ab", reduction.key = "AbTSNE_")
cl_mono[["Ab_snn"]] <- FindNeighbors(Ab.dist)$snn
tsne_feat_cl_mono  <- FeaturePlot(cl_mono, reduction = 'tsne_Ab', label = TRUE, pt.size = 1, features = markers_ab[c(3, 6, 8, 11, 14, 15, 16)])
ggsave(tsne_feat_cl_mono, file="output/tSNE_overlay_cl_monos.png")

In [ ]:
cl_mono <- FindClusters(cl_mono, resolution = 0.3, graph.name = "Ab_snn")

tsne_cl_mono  <- DimPlot(cl_mono, reduction = 'tsne_Ab', label = TRUE, pt.size = 1)
ggsave(tsne_cl_mono, file="output/tsne_cluster_cl_monos.png")

cl_mono  <- RenameIdents(cl_mono, "2" = "CD14dim", "4" = "CD14dim")

In [ ]:
# project cd14dim cluster on original umap
umap_mapping  <- DimPlot(phagocytes, reduction = 'umap', label = FALSE, pt.size = 1, cols = cols_phagocytes, group.by = "cell.annotation") + 
    theme(legend.position = "none") + 
      geom_density2d(data= 
      data.frame(Embeddings(subset(cl_mono, idents = "CD14dim")[["umap"]])), 
      aes(x = UMAP_1, y = UMAP_2), col="red", bins = 8)

umap_mapping   
ggsave(umap_mapping, file ="output/umap_mapping_selected.png")

In [ ]:
# add CD14dim to clustering annotation
phagocytes@meta.data$cell.annotation.CD14dim <- as.character(phagocytes@meta.data$cell.annotation)

CD14dim_names  <- subset(cl_mono, idents = "CD14dim") %>% colnames
phagocytes@meta.data[rownames(phagocytes@meta.data) %in% CD14dim_names,"cell.annotation.CD14dim"]  <- "CD14dim"
phagocytes@meta.data$cell.annotation.CD14dim  <- factor(phagocytes@meta.data$cell.annotation.CD14dim)

In [ ]:
# plot heatmap for surface markers
Ab_data_phagocytes  <- GetAssayData(phagocytes, slot = "data", assay = "Ab") %>% 
    data.frame(check.names = F) %>% 
    t() %>% data.frame %>% cbind(phagocytes@meta.data$cell.annotation.CD14dim)
colnames(Ab_data_phagocytes)[22]  <- "cell.annotation.CD14dim"

heat  <- Ab_data_phagocytes %>% group_by(cell.annotation.CD14dim) %>% summarise_all(mean)

heat_m  <- data.matrix(heat[-1])
rownames(heat_m)  <- heat$cell.annotation.CD14dim
colnames(heat_m)  <- str_replace(colnames(heat_m), ".Ab", "")

white.black <- colorRampPalette(c("white", "black"))(n = 9)

pheatmap(mat = heat_m[,1:18], 
         scale= "none",
         color = white.black,
        #  breaks = breaks,
         cluster_rows=F,
         cluster_cols=F,
        #  annotation_row = annotation,
        #  annotation_colors =  ann_col,
         cellwidth = 10, cellheight = 10,
         border_color = NA,
         filename= "output/phagocytes_heatmap_ab.pdf"
         )

## Find Markers that differentiate selected node from other clusters

In [ ]:
Idents(phagocytes)  <- phagocytes@meta.data$cell.annotation.CD14dim
DefaultAssay(phagocytes)  <- "RNA"

phagocyte.markers <- phagocytes %>% subset(idents = c("CD14dim", "Classical monocytes", "Non-classical monocytes", "Intermediate monocytes", "Dendritic cells")) %>% 
    FindAllMarkers(only.pos = FALSE, min.pct = 0.1, logfc.threshold = 0.25)

top_phagocyte_markers  <- phagocyte.markers %>%
    group_by(cluster) %>%
    slice_max(n = 15, order_by = avg_log2FC)

dotplot_phagocytes_selected  <- phagocytes %>% subset(idents = c("CD14dim", "Classical monocytes", "Non-classical monocytes", "Intermediate monocytes", "Dendritic cells")) %>% 
    DotPlot(features = unique(top_phagocyte_markers$gene), cols = c("white", "#557F7A"), cluster.idents = T) + RotatedAxis()
ggsave(dotplot_phagocytes_selected, file = "output/dotplot_phagocytes_differential_genes_CD14dim.pdf", height = 2.5, width = 19)


In [ ]:
# DEGs for all clusters
diff_markers  <- function(data, cell_type){
        diff  <- subset(data, subset = (twin_pair != "none" & cell.annotation.CD14dim == cell_type)) %>% 
                        FindMarkers(., test.use="LR", latent.vars = "twin_pair", group.by = "diagnosis", ident.1 = "MS", ident.2 = "Healthy", logfc.threshold = 0, features = features) %>%
                        mutate(cell.type = cell_type, gene = rownames(.))
    }

diff_markers_res  <- unique(Idents(phagocytes)) %>% map_dfr(~ diff_markers(data = phagocytes, cell_type = .))
sig_markers  <- diff_markers_res %>% subset(p_val_adj <.05)

write.csv(sig_markers, file = "output/DEG_all_clusters.csv")

# plot volcanos per cluster
plot_volcano  <- function(data, cell_type){
    volcano <- ggplot(subset(data, cell.type == cell_type), aes(x = avg_log2FC, y = -log10(p_val_adj))) +        
    geom_vline(xintercept = 0) +
    geom_hline(yintercept = -log10(0.05), color ="grey", linetype ="dashed") +
    geom_point(data = subset(data, cell.type == cell_type),
                color = "grey", alpha = 0.5) +
    geom_point(data = subset(data, cell.type == cell_type & avg_log2FC > 0 & p_val_adj < .05)[1:20,],
                fill = "#CC242A", alpha = 1, shape=21, size= 2.5) +
    geom_point(data = subset(data, cell.type == cell_type & avg_log2FC < 0 & p_val_adj < .05)[1:20,],
                fill = "#557F7A", alpha = 1, shape=21, size= 2.5) +    
    geom_text_repel(data=subset(data, cell.type == cell_type & p_val_adj < .05)[1:20,], aes(label = gene))+
    theme_linedraw() + 
    theme(panel.grid = element_blank(), legend.position = "none") +
    xlab("log2(average fold change)") +
    ylab("-log10(p-value)") 
    volcano
    ggsave(volcano, file=paste0("output/volcano_", cell_type, ".pdf"), height = 4, width = 5)
    }

unique(Idents(phagocytes)) %>% map(~ plot_volcano(data = diff_markers_res, cell_type = .))

In [ ]:
# GSEA for functional differences between twins
LR_twins_meta  <- function(data, cell_type, features){
    seurat_subset  <- subset(data, subset = (twin_pair != "none" & cell.annotation.CD14dim == cell_type))
    model_data  <- seurat_subset[[c(features, "diagnosis", "twin_pair")]]
    model_data$diagnosis  <- factor(model_data$diagnosis, levels = c("Healthy", "MS"), labels = c(0, 1))
    fmla  <- as.formula(object = paste("diagnosis ~", features, "+ twin_pair"))
    fmla2  <- as.formula(object = "diagnosis ~ twin_pair")
    model1 <- glm(formula = fmla, data = model_data, family = "binomial")
    model2 <- glm(formula = fmla2, data = model_data, family = "binomial")
    lr_res  <- lrtest(model1, model2)
    res  <- c(model1$coefficients, lr_res$Pr[2])
}

calculate_logFC  <- function(data, cell_type, features){
    seurat_subset  <- subset(data, subset = (twin_pair != "none" & cell.annotation.CD14dim == cell_type))
    feature_data  <- seurat_subset[[c(features, "diagnosis", "twin_pair")]]
    means  <- feature_data %>% group_by(diagnosis) %>% summarize_at(features, mean)
    fc  <- (subset(means, diagnosis == "MS")[,2] - subset(means, diagnosis == "Healthy")[,2])
}

reverselog_trans <- function(base = exp(1)) {
    trans <- function(x) -log(x, base)
    inv <- function(x) base^(-x)
    trans_new(paste0("reverselog-", format(base)), trans, inv, 
              log_breaks(base = base), 
              domain = c(1e-100, Inf))
}

CSF2RA signaling module
geneset access via https://maayanlab.cloud/Harmonizome/gene_set/CSF2RA/Pathway+Commons+Protein-Protein+Interactions

In [ ]:
CSF2_genes  <- read.delim("CSF2_geneset.txt", header = F)[,1]
phagocytes <- AddModuleScore(
  object = phagocytes,
  features = list(CSF2_genes),
  ctrl = 100,
  name = 'CSF2_genes'
)

# heatmap CSF2 signalling
seurat_sub  <- subset(phagocytes, subset = cell.annotation.CD14dim == "CD14dim")

data_csf2  <- data.frame(CSF2_genes1 = seurat_sub[["CSF2_genes1"]],twin = seurat_sub@meta.data$twin, sample = seurat_sub@meta.data$sample, 
        diagnosis = seurat_sub@meta.data$diagnosis, twin_pair = seurat_sub@meta.data$twin_pair)

data_csf2_sub  <- subset(data_csf2, twin_pair != "none" & twin_pair != 4)

heat  <- data_csf2_sub %>% group_by(sample) %>% summarise(mean(CSF2_genes1)) %>% merge(md) %>%
    arrange(twin_pair, diagnosis)
mat  <- matrix(data = (heat %>% pull(2)), ncol = (2), byrow = T, dimnames = list(paste("twin pair", unique(heat$twin_pair))))

pheatmap(mat = mat, 
          scale= "none",
           color = colorRampPalette(c("white", "#557F7A"))(n = 9),
          cluster_rows=F,
          cluster_cols=F,
          cellwidth = 10, cellheight = 10,
          border_color = NA,
          filename= "output/CSF2_signalling_heatmap_CD14_dim.pdf"
    )

# heatmaps for individual genes involved in CSF2RA signaling
seurat_sub  <- subset(phagocytes, subset = (twin_pair != "none" & cell.annotation.CD14dim == "CD14dim" & twin_pair !=4))
sub_counts  <- GetAssayData(seurat_sub, slot = "data", assay = "RNA") %>% data.frame(check.names = F) %>% t() %>% 
            data.frame %>% cbind(seurat_sub[[c("sample", "diagnosis", "twin_pair")]])

data_csf2ra  <- data.frame(CSF2RA = sub_counts[,"CSF2RA"], sample = sub_counts$sample, 
        diagnosis = sub_counts$diagnosis, twin_pair = sub_counts$twin_pair)


heat  <- data_csf2ra %>% group_by(sample) %>% summarise(mean(CSF2RA)) %>% merge(md) %>%
    arrange(twin_pair, diagnosis)
mat  <- matrix(data = (heat %>% pull(2)), ncol = (2), byrow = T, dimnames = list(paste("twin pair", unique(heat$twin_pair))))


pheatmap(mat = mat, 
          scale= "none",
           color = white.black,
          cluster_rows=F,
          cluster_cols=F,
          cellwidth = 10, cellheight = 10,
          border_color = NA,
          filename= "output/CSF2RA_heatmap_CD14_dim.pdf"
)

In [ ]:
#Stat 5 signaling
data_STAT5A  <- data.frame(STAT5A = sub_counts[,"STAT5A"], sample = sub_counts$sample, 
        diagnosis = sub_counts$diagnosis, twin_pair = sub_counts$twin_pair)

heat  <- data_STAT5A %>% group_by(sample) %>% summarise(mean(STAT5A)) %>% merge(md) %>%
    arrange(twin_pair, diagnosis)
mat  <- matrix(data = (heat %>% pull(2)), ncol = (2), byrow = T, dimnames = list(paste("twin pair", unique(heat$twin_pair))))

pheatmap(mat = mat, 
          scale= "none",
           color = white.black,
          # breaks = breaks,
          cluster_rows=F,
          cluster_cols=F,
          #  annotation_row = annotation,
          #  annotation_colors =  ann_col,
          cellwidth = 10, cellheight = 10,
          border_color = NA,
          filename= "output/STAT5A_heatmap_CD14_dim.pdf"
    )

data_STAT5B  <- data.frame(STAT5B = sub_counts[,"JAK2"], sample = sub_counts$sample, 
        diagnosis = sub_counts$diagnosis, twin_pair = sub_counts$twin_pair)


heat  <- data_STAT5B %>% group_by(sample) %>% summarise(mean(STAT5B)) %>% merge(md) %>%
    arrange(twin_pair, diagnosis)
mat  <- matrix(data = (heat %>% pull(2)), ncol = (2), byrow = T, dimnames = list(paste("twin pair", unique(heat$twin_pair))))


pheatmap(mat = mat, 
          scale= "none",
           color = white.black,
          # breaks = breaks,
          cluster_rows=F,
          cluster_cols=F,
          #  annotation_row = annotation,
          #  annotation_colors =  ann_col,
          cellwidth = 10, cellheight = 10,
          border_color = NA,
          filename= "output/JAK2_heatmap_CD14_dim.pdf"
    )
dev.off()

Type 1 IFN signalling
gene set: https://www.gsea-msigdb.org/gsea/msigdb/geneset_page.jsp?geneSetName=REACTOME_INTERFERON_ALPHA_BETA_SIGNALING&keywords=type%201%20interferon


In [ ]:
IFN1_genes  <- read.delim("IFN1_geneset.txt", header = F)[,1]

phagocytes <- AddModuleScore(
  object = phagocytes,
  features = list(IFN1_genes),
  ctrl = 100,
  name = 'IFN1_genes'
)

LR_res_IFN1 <- unique(Idents(phagocytes)) %>% map_dfr(~ LR_twins_meta(data = phagocytes, cell_type = ., features = "IFN1_genes1")) %>% mutate(cell.type = unique(Idents(phagocytes)))
colnames(LR_res_IFN1)[10]  <- "p_val"
write.csv(LR_res_IFN1, file = "output/IFN1_signature_stats.csv")

IFN1_violin  <- phagocytes %>% subset(subset = (twin_pair != "none" & cell.annotation.CD14dim %in% subset(LR_res_IFN1, p_val < 0.05)$cell.type[1:3])) %>% 
        VlnPlot(features = "IFN1_genes1", group.by = "cell.annotation.CD14dim", split.by = "diagnosis", cols = c("#557F7A", "#CC242A"), pt.size = 0.01)
IFN1_violin  <- delete_layers(IFN1_violin, "GeomViolin") + geom_violin(scale="area", aes(fill=split), alpha=1, draw_quantiles = .5) + xlab("") + ylab("Module score") + ggtitle("Type 1 IFN signature")

ggsave(IFN1_violin, file = "output/IFN1_signature_violin_test.pdf", width = 5, height = 5)

compare genes involved in phagocytosis, antigen presentation and ROS production between clustersphagocytosis
gene set from: https://www.gsea-msigdb.org/gsea/msigdb/cards/KEGG_FC_GAMMA_R_MEDIATED_PHAGOCYTOSIS


In [ ]:
phago_genes  <- read.delim("Phagocytosis_geneset.txt", header = F)[,1]

phagocytes <- AddModuleScore(
  object = phagocytes,
  features = list(phago_genes),
  ctrl = 100,
  name = 'Phago_genes'
)

ROS gene set from: https://maayanlab.cloud/Harmonizome/gene_set/reactive+oxygen+species+metabolic+process/GO+Biological+Process+Annotations


In [ ]:
ROS_genes  <- read.delim("ROS_geneset.txt", header = F)[,1]

phagocytes <- AddModuleScore(
  object = phagocytes,
  features = list(ROS_genes),
  ctrl = 100,
  name = 'ROS_genes'
)

antigen presentation gene set from: https://maayanlab.cloud/Harmonizome/gene_set/antigen+processing+and+presentation/KEGG+Pathways


In [ ]:
AP_genes  <- read.delim("ROS_geneset.txt", header = F)[,1]

phagocytes <- AddModuleScore(
  object = phagocytes,
  features = list(AP_genes),
  ctrl = 100,
  name = 'AP_genes'
)

In [ ]:
# plot the module scores
Phago_violin  <- phagocytes %>% subset(subset = (twin_pair != "none" & cell.annotation.CD14dim %in% c("Classical monocytes", "Non-classical monocytes", "Intermediate monocytes", "Dendritic cells", "CD14dim"))) %>% 
        VlnPlot(features = "Phago_genes1", group.by = "cell.annotation.CD14dim", cols = cols_phagocytes)
Phago_violin  <- delete_layers(Phago_violin, "GeomViolin") + geom_violin(scale="area", aes(fill=ident), alpha=1, draw_quantiles = .5) + xlab("") + ylab("Module score") + ggtitle("Phagocytosis")
ggsave(Phago_violin, file = "output/Phagocyte_signature_violin_clusters.pdf", width = 5, height = 5)


AP_violin  <- phagocytes %>% subset(subset = (twin_pair != "none" & cell.annotation.CD14dim %in% c("Classical monocytes", "Non-classical monocytes", "Intermediate monocytes", "Dendritic cells", "CD14dim"))) %>% 
        VlnPlot(features = "AP_genes1", group.by = "cell.annotation.CD14dim", cols = cols_phagocytes)
AP_violin  <- delete_layers(AP_violin, "GeomViolin") + geom_violin(scale="area", aes(fill=ident), alpha=1, draw_quantiles = .5) + xlab("") + ylab("Module score") + ggtitle("Antigen Presentation")
ggsave(AP_violin, file = "output/AP_signature_violin_clusters.pdf", width = 5, height = 5)

ROS_violin  <- phagocytes %>% subset(subset = (twin_pair != "none" & cell.annotation.CD14dim %in% c("Classical monocytes", "Non-classical monocytes", "Intermediate monocytes", "Dendritic cells", "CD14dim"))) %>% 
        VlnPlot(features = "ROS_genes1", group.by = "cell.annotation.CD14dim", cols = cols_phagocytes)
ROS_violin  <-  delete_layers(ROS_violin, "GeomViolin") + geom_violin(scale="area", aes(fill=ident), alpha=1, draw_quantiles = .5) + xlab("") + ylab("Module score") + ggtitle("ROS")
ggsave(ROS_violin, file = "output/ROS_signature_violin_clusters.pdf", width = 5, height = 5)
